# Applied Deep Learning
> Applied Deep Learning [Course](https://github.com/maziarraissi/Applied-Deep-Learning) 

- toc: true 
- badges: true
- comments: true
- categories: [jupyter,deeplearning,python]


## Deep Learning overview
* we can look at deep learning as an algorithm that writes algorithms, like a compiler
 - in this case the source code would be the data: (examples/experiences)
 - excutable code would be the deployable model 

 * Deep: Functions compositions  $ f_l f_{l-1} .... f_1$
 * Learning: Loss, Back-propagation, and Gradient Descent

 * $ L(\theta) \approx J(\theta)$ --> noisy estimate of the objective function due to mini-batching. That's why we call it stochastic Gradient Descent 
 * why do we use the first order derivative, not the second order one (the hessian), because order of first order derivative is N, but for the hessian it's N*N, so it's computationally expensive and slow 
 ### Optimizers
 * to make gradient descent faster, we can add momentum to it.
  * another way is to use Nesttov Accelerated Gradient: the idea is to look ahead while computing the gradient, so we can add that to the momentum




  * RMSprop: A mini-batch version of rprop method. the original rprop can't work with mini batches, as it doesn't consider the magnitude of the gradient, but only the sign of it, and it would multiply the gradient by a fixed factor every time depending on the sign. 

  ![](assets/Applied_deep_learning/rprop.png)


  * Nestrov adaptive optimizer: the main idea is that we know that we gonna update the weights according to our average velocity so far, and also our gradient, but this can cause us to over shoot as we have a huge velocity moving down the hill, so why not update first the weights according to our velocity and see where that gets us (the look ahead term), and then we update the weights according to the gradient there 

* Adam:
    - can take different time steps for each paramater (Adaptive steps) (took concepts from Adadelta)
    - can also has momentum for all parameter wich can lead to faster convergence
* Nadam: Just like Adam but with added nestrov acceleration look ahead functionality so we can slow down we go near the goal

## ImageNet:



## Dropout
* A simple method to prevent the NN from overfitting 
* CNNS are less prune to overfitting becaue the weight sharing idea, that we have a set of filters fot the entire image 
* you can look at dropout as a smart way of ensembling, as it combines exponentially many different networks architectures effienctly. 


### Network In Network
* the main idea is to put a network inside another network

* they introduced multi layer preceptron conv layer which is a conv layer followed by a few FC layers
* this idea is bisacally a (one to one convution) 
* they introduced a global averaging pooling: insted of adding a bunch of FC layers at the end of teh conv architecture, we can just average multible channels  from the last conv layer to form the output layer 

* one by one convolution is a normal convolution with fliter size of 1 by 1 

### VGG Net

#### Local Response Normalization:
* the idea is to normalize a pixel acros nearing channels 